In [1]:
from google.colab import drive
drive.mount('/content/drive') 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
import os
import pickle
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import Model
import time
import matplotlib.pyplot as plt
import random

In [3]:
os.chdir("/content/drive/My Drive/Anomaly detection - BuildSys2020/")

In [4]:
# For reproducibility
np.random.seed(12)
tf.random.set_seed(12)
random.seed(1)

In [5]:
def usable_features(df_building, features_used):  # for droping the features not to be considered in the model
  ## df_building - pandas dataframe of the datset
  ## features_used - features to be included in the cleaned dataset

  # creating list of columns that are not used 
  col = df_building.columns
  features_not_used = [] # store columns that won't be used
  for x in col:
    if x not in features_used:
      features_not_used.append(x)

  # drop the columns
  df_building= df_building.drop(features_not_used, axis = 1) # dropping not used features 

  # to fill NA data we simply replace the values with 0
  df_building = df_building.fillna(0)
  
  return df_building

In [6]:
def normalize_minmax(df, col):  # function to normalize the columns 
    ## df - data frame containing the columns
    ## columns to be normalized  
    result = df.copy()
    for feature_name in col:
        max_value = df[feature_name].max()
        min_value = df[feature_name].min()

        print("Feature Name : {}  :  Max Value - {} ; Min Value - {}".format(feature_name, max_value, min_value))
        # normalize 
        result[feature_name] = (df[feature_name] - min_value) / (max_value - min_value)
    return result 

In [7]:
def normalize_zAlgo(df, col):  # function to normalize the columns 
    ## df - data frame containing the columns
    ## columns to be normalized  
    result = df.copy()
    for feature_name in col:
        std_value = df[feature_name].std()
        mean_value = df[feature_name].mean()

        print("Feature Name : {}  :  Std Value - {} ; Mean Value - {}".format(feature_name, std_value, mean_value))
        # normalize 
        result[feature_name] = (df[feature_name] - mean_value) / std_value
    return result

In [8]:
def clean_dataset_ashrae(dataset, meter_types, features_used, z_columns, minmax_columns):
  ## meter_types - list of meters to be included 
  ## features_used - list of features to be included in the final data
  ## z_columns - list of columns to be normalized with Z algorithm
  ## minmax_columns - list of columns to be normalized with min-max normalization

  meter_dataset = dataset.loc[dataset['meter']==0]
  
  # create a new column  
  meter_dataset['meter_reading_log'] = np.log(meter_dataset['meter_reading']+1) 

  #convert categorical data type to int
  meter_dataset['primary_use'] = meter_dataset['primary_use'].astype('category')
  unique_primUse_list = list(meter_dataset.primary_use.unique())
  unique_primUse_dict = {unique_primUse_list[i]: i for i in range(0, len(unique_primUse_list))}
  meter_dataset['primary_use'] = meter_dataset['primary_use'].map(unique_primUse_dict).astype(int)

  # remove columns that won't be included in the model's input 
  df_train = usable_features(meter_dataset, features_used)

  # normalize features 
  df_train_norm1 = normalize_zAlgo(df_train, z_columns) # z-algorithm
  df_train_norm = normalize_zAlgo(df_train_norm1, minmax_columns) # min-max scaler

  print("df_train_norm shape {}".format(df_train_norm.shape))
  return df_train_norm

In [9]:
## creating input sequence of length - seq_length
def create_windows(arr_data, arr_anomaly_label, arr_op, seq_length, nb_features):
  ## arr_data - normalized dataset as a numpy array 
  ## arr_op - array containing anomaly labels for each time step 
  ## seq_length - wiindow length 
  ## nb_features - no. of features to be used as input 

  anom_x = [] # storing anomalous windows - input 
  non_anom_x = [] # storing non-anomalous windows - input 
  anom_y = [] # storing anomalous windows - output 
  non_anom_y = [] # storing non-anomalous windows - output 

  # run a loop to move a seq_length size window across data non-overlapping
  for i in range(0,arr_data.shape[0]//seq_length):

    # slice the window 
    window_features = arr_data[i*seq_length:(i+1)*seq_length].reshape((seq_length, nb_features))   # window of seq_length
    window_output = arr_op[i*seq_length:(i+1)*seq_length].reshape((seq_length,1))

    is_anomaly = np.count_nonzero(arr_anomaly_label[i*seq_length:(i+1)*seq_length])  #if even at one time point anomaly is present the window would be considered anomalous
    # print(is_anomaly)

    if is_anomaly > 0 :  # separating the anomalous and non-anomalous data 
      anom_x.append(window_features)
      anom_y.append(window_output)
    else:
      non_anom_x.append(window_features)
      non_anom_y.append(window_output)

  return non_anom_x, non_anom_y, anom_x, anom_y 

In [10]:
def mix_data(data_x, data_y):
  #  Mix Data (to make it similar to i.i.d)
  data_idx = np.random.permutation(len(data_x))


  output_data_x = []  # Store shuffled data
  output_data_y = []

  for i in range(len(data_x)):
    output_data_x.append(data_x[data_idx[i]])
    output_data_y.append(data_y[data_idx[i]])

  print("ouput_x shape {} , {}".format(len(output_data_x), output_data_x[0].shape))
  return output_data_x, output_data_y

In [11]:
with open('ashrae_rank1_train_clean.pkl', 'rb') as f:
      dataset = pickle.load(f)

In [12]:
def load_data_ashrae(dataset, meter_types, features_used, z_columns, minmax_columns, seq_length, nb_buildings, output_feature):
  ## seq_length - window length 
  ## nb_buildings - no of buildings for the input samples 
  ## output_feature - feature that would be taken as the output to be predicted 

  ## import the dataset 
  

  df_final_train_norm = clean_dataset_ashrae(dataset, meter_types, features_used, z_columns, minmax_columns)

  # lists to store final input data  
  anom_x = []
  anom_y = []
  non_anom_x = []
  non_anom_y = []

  # group data on the basis of their building_id 
  grp_data = df_final_train_norm.groupby('building_id')

  # create a random list of building whose data would be considered 
  list_buildings = random.sample(list(grp_data.groups.keys()), nb_buildings)

  # loop throught the building ids - segregate them into Anomalous and Non-Anomalouws windows
  for grp_no in list_buildings:

    # pick a building id whose data needs to be added into final input data
    building_data = grp_data.get_group(grp_no)
    #building_id is not needed anymore 
    building_data = building_data.drop('building_id', axis = 1) # dropping not used features 

    # separating labels from the data 
    arr_labels = building_data['is_bad_meter_reading'].to_numpy()
    # label is not not need anymore 
    building_data = building_data.drop('is_bad_meter_reading', axis = 1) # dropping not used features 

    # separating output_feature from the data 
    arr_output = building_data[output_feature].to_numpy()
    # output_feature is not not need anymore 
    building_data = building_data.drop(output_feature, axis = 1) # dropping not used features 

    # making sure the data is sequential 
    building_data.sort_values("timestamp", axis = 0, ascending = True) 
    building_data = building_data.drop('timestamp', axis = 1) # dropping not used features 

    # creating numpy array of remaining features 
    building_data = building_data.to_numpy()

    # creating windowed data
    nb_features = building_data.shape[1]
    na_x, na_y, a_x, a_y = create_windows(building_data, arr_labels, arr_output, seq_length, nb_features)

    print(" na_x - len {}, shape {} ".format(len(na_x), na_x[0].shape))
    print(" a_x - len {}".format(len(a_x)))
    # accumulating single bulding data 
    anom_x.extend(a_x)
    anom_y.extend(a_y)
    non_anom_x.extend(na_x)
    non_anom_y.extend(na_y)

  # making data similar to i.i.d.
  anom_x, anom_y = mix_data(anom_x, anom_y)
  non_anom_x, non_anom_y = mix_data(non_anom_x, non_anom_y)

  return non_anom_x, non_anom_y, anom_x, anom_y 

In [13]:
## change these lists as per model's input 
meter_types = [0] 

features_used = ['building_id','timestamp', 'air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 
                  'sea_level_pressure', 'wind_speed','primary_use', 'square_feet', 'year_built', 'floor_count', 
                  'hour', 'weekday', 'month', 'is_bad_meter_reading','meter_reading_log']

z_columns = ['air_temperature', 'cloud_coverage', 'dew_temperature', 'precip_depth_1_hr', 'sea_level_pressure', 'wind_speed', 
              'square_feet', 'year_built' ]          

minmax_columns = ['hour', 'month', 'weekday']

seq_length = 24 


nb_buildings = 145

output_feature = 'meter_reading_log'

In [14]:
non_anom_x, non_anom_y, anom_x, anom_y = load_data_ashrae(dataset, meter_types, features_used, z_columns, minmax_columns, seq_length, nb_buildings, output_feature)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the 

Feature Name : air_temperature  :  Std Value - 10.692255020141602 ; Mean Value - 15.730816841125488
Feature Name : cloud_coverage  :  Std Value - 125.52488356038128 ; Mean Value - 114.02984235849534
Feature Name : dew_temperature  :  Std Value - 10.18441104888916 ; Mean Value - 8.148475646972656
Feature Name : precip_depth_1_hr  :  Std Value - 6.967662172348021 ; Mean Value - 0.21803520629869555
Feature Name : sea_level_pressure  :  Std Value - 18.378942489624023 ; Mean Value - 1023.1770629882812
Feature Name : wind_speed  :  Std Value - 2.313711404800415 ; Mean Value - 3.6503612995147705
Feature Name : square_feet  :  Std Value - 112109.99461909568 ; Mean Value - 92714.31195025914
Feature Name : year_built  :  Std Value - 95.44950642767324 ; Mean Value - 168.47074200868758
Feature Name : hour  :  Std Value - 6.922763943175829 ; Mean Value - 11.500643317958595
Feature Name : month  :  Std Value - 3.4435748699115054 ; Mean Value - 6.552057846381409
Feature Name : weekday  :  Std Value -

In [15]:
len(non_anom_y)

46182

In [16]:
# int((len(non_anom_y)*train_percent)) + 4796 + 4796

In [17]:
def split_data(train_percent, val_percent, test_percent, output_non_anom_x, output_non_anom_y ):

  # train dataset
  X_train_non_anom = output_non_anom_x[:int((len(output_non_anom_y)*train_percent))]
  Y_train_non_anom = output_non_anom_y[:int((len(output_non_anom_y)*train_percent))]

  # validation dataset
  X_val_non_anom = output_non_anom_x[int((len(output_non_anom_y)*train_percent)):-int((len(output_non_anom_y)*test_percent) )]
  Y_val_non_anom = output_non_anom_y[int((len(output_non_anom_y)*train_percent)):-int((len(output_non_anom_y)*test_percent) )]

  # test dataset                                                          
  X_test_non_anom = output_non_anom_x[-int(len(output_non_anom_y)*test_percent):]
  Y_test_non_anom = output_non_anom_y[-int(len(output_non_anom_y)*test_percent):]     

  return  X_train_non_anom, Y_train_non_anom, X_val_non_anom, Y_val_non_anom, X_test_non_anom, Y_test_non_anom

In [18]:
train_percent = 0.8
val_percent = 0.1
test_percent = 0.1

In [19]:
X_train_non_anom, Y_train_non_anom, X_val_non_anom, Y_val_non_anom, X_test_non_anom, Y_test_non_anom = split_data(train_percent, val_percent, test_percent, non_anom_x, non_anom_y)

In [20]:
!pip install pyod

     |████████████████████████████████| 102kB 2.5MB/s 
     |████████████████████████████████| 2.1MB 5.3MB/s 
  Created wheel for pyod: filename=pyod-0.8.1-cp36-none-any.whl size=105653 sha256=de0baeb05ca2c13fb0eb787be941a1ea4c56013760f0913ea45f8db2b55d62d7
  Stored in directory: /root/.cache/pip/wheels/2e/ca/18/727e9d98a41f5f4385a97d5b429f3a9c8fbee13f9780c18642
  Created wheel for combo: filename=combo-0.1.1-cp36-none-any.whl size=42111 sha256=b044ec4923a49a1e96134bce8230ed35bea5fe0f980fc00b8fd65a6e4e0e7825
  Stored in directory: /root/.cache/pip/wheels/55/ec/e5/a2331372c676c467e70c6646e646edf6997d5c4905b8c0f5e6
  Created wheel for suod: filename=suod-0.0.4-cp36-none-any.whl size=2167157 sha256=51559e4b3d2ee321adeac50e58fdf38021a164716f2da8d8daf2bc0e70afcfbb
  Stored in directory: /root/.cache/pip/wheels/57/55/e5/a4fca65bba231f6d0115059b589148774b41faea25b3f2aa27
Successfully built pyod combo suod


In [21]:
from pyod.models.knn import KNN   # kNN detector
from pyod.utils import data

In [22]:
contamination = 0.1  # percentage of outliers
n_train = 200  # number of training points
n_test = 100  # number of testing points

X_train, y_train, X_test, y_test = data.generate_data(
    n_train=n_train, n_test=n_test, contamination=contamination)

/usr/local/lib/python3.6/dist-packages/pyod/utils/data.py:190: FutureWarning: behaviour="old" is deprecated and will be removed in version 0.8.0. Please use behaviour="new", which makes the returned datasets in the order of X_train, X_test, y_train, y_test.
  FutureWarning)


In [23]:
X_train.shape

(200, 2)

In [24]:
len(Y_train_non_anom)

36945

In [30]:
anom_y[0].shape

(24, 1)

In [38]:
def create_data_pyod(Y_train_non_anom, anom_y, Y_test_non_anom):

  # Anomalous  - train data
  arr_1 = np.zeros((1,24))
  for i in range(len(anom_y)):
    arr_1 = np.concatenate([arr_1, anom_y[i].reshape(1, 24)], axis=0)
  arr_1 = np.delete(arr_1, (0), axis=0)

  # Non- Anomalous  - train data
  arr_0 = np.zeros((1,24))
  for i in range(len(Y_train_non_anom)):
    arr_0 = np.concatenate([arr_0, Y_train_non_anom[i].reshape(1, 24)], axis=0)
  arr_0 = np.delete(arr_0, (0), axis=0)

  X_train = np.concatenate([arr_0, arr_1[:4000]], axis=0)

  Y_train = np.zeros((arr_0.shape[0])) 
  Y_train = np.concatenate([Y_train, np.ones((4000))], axis=0)

  # Non- anomalous- test data 
  test_arr_0 = np.zeros((1,24))
  for i in range(len(Y_test_non_anom)):
    test_arr_0 = np.concatenate([test_arr_0, Y_test_non_anom[i].reshape(1, 24)], axis=0)
  test_arr_0 = np.delete(test_arr_0, (0), axis=0)

  # Anomalous- test data 
  X_test = np.concatenate([test_arr_0, arr_1[4000:]], axis=0)

  Y_test = np.zeros((test_arr_0.shape[0]))  
  Y_test = np.concatenate([Y_test, np.ones((len(anom_y[4000:])))], axis=0)

  return X_train, Y_train, X_test, Y_test, arr_0, arr_1

In [39]:
X_train, Y_train, X_test, Y_test, arr_0, arr_1 = create_data_pyod(Y_train_non_anom, anom_y, Y_test_non_anom)

In [40]:
from sklearn.utils import shuffle
x_train, y_train = shuffle(X_train, Y_train)
x_test, y_test = shuffle(X_test, Y_test)

In [41]:
# train kNN detector
clf_name = 'KNN'
clf = KNN()
clf.fit(X_train)

KNN(algorithm='auto', contamination=0.1, leaf_size=30, method='largest',
  metric='minkowski', metric_params=None, n_jobs=1, n_neighbors=5, p=2,
  radius=1.0)

In [42]:
# get the prediction labels and outlier scores of the training data
y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
y_train_scores = clf.decision_scores_  # raw outlier scores

# get the prediction on the test data
y_test_pred = clf.predict(X_test)  # outlier labels (0 or 1)
y_test_scores = clf.decision_function(X_test)  # outlier scores

In [43]:
"""Compare all detection algorithms 
"""
# Author: Yue Zhao <zhaoy@cmu.edu>
# License: BSD 2 clause

from __future__ import division
from __future__ import print_function

import os
import sys

# supress warnings for clean output
import warnings

warnings.filterwarnings("ignore")
from numpy import percentile

# Import all models
from pyod.models.abod import ABOD
from pyod.models.cblof import CBLOF
from pyod.models.feature_bagging import FeatureBagging
from pyod.models.hbos import HBOS
from pyod.models.iforest import IForest
from pyod.models.knn import KNN
from pyod.models.lof import LOF
from pyod.models.loci import LOCI
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.pca import PCA
from pyod.models.sos import SOS
from pyod.models.lscp import LSCP
from pyod.models.cof import COF
from pyod.models.sod import SOD


In [44]:
# Define the number of inliers and outliers
n_samples = y_train.shape[0]
outliers_fraction = arr_1.shape[0] / arr_0.shape[0]

n_inliers = arr_0.shape[0]
n_outliers = arr_1.shape[0]

ground_truth = y_train

clusters_separation = [0]

# initialize a set of detectors for LSCP
detector_list = [LOF(n_neighbors=5), LOF(n_neighbors=10), LOF(n_neighbors=15),
                 LOF(n_neighbors=20), LOF(n_neighbors=25), LOF(n_neighbors=30),
                 LOF(n_neighbors=35), LOF(n_neighbors=40), LOF(n_neighbors=45),
                 LOF(n_neighbors=50)]

# Show the statics of the data
print('Number of inliers: %i' % n_inliers)
print('Number of outliers: %i' % n_outliers)
print(
    'Ground truth shape is {shape}. Outlier are 1 and inliers are 0.\n'.format(
        shape=ground_truth.shape))
print(ground_truth, '\n')

random_state = 42

Number of inliers: 36945
Number of outliers: 4859
Ground truth shape is (40945,). Outlier are 1 and inliers are 0.

[0. 0. 1. ... 0. 0. 0.] 



In [45]:
# Define nine outlier detection tools to be compared
classifiers = {
    # 'Angle-based Outlier Detector (ABOD)':
    #     ABOD(contamination=outliers_fraction),
    'Cluster-based Local Outlier Factor (CBLOF)':
        CBLOF(contamination=outliers_fraction,
              check_estimator=False, random_state=random_state),
    'Feature Bagging':
        FeatureBagging(LOF(n_neighbors=35),
                       contamination=outliers_fraction,
                       random_state=random_state),
    'Histogram-base Outlier Detection (HBOS)': HBOS(
        contamination=outliers_fraction),
    'Isolation Forest': IForest(contamination=outliers_fraction,
                                random_state=random_state),
    'K Nearest Neighbors (KNN)': KNN(
        contamination=outliers_fraction),
    'Average KNN': KNN(method='mean',
                       contamination=outliers_fraction),
    # 'Median KNN': KNN(method='median',
    #                   contamination=outliers_fraction),
    'Local Outlier Factor (LOF)':
        LOF(n_neighbors=35, contamination=outliers_fraction),
    # 'Local Correlation Integral (LOCI)':
    #     LOCI(contamination=outliers_fraction),
    'Minimum Covariance Determinant (MCD)': MCD(
        contamination=outliers_fraction, random_state=random_state),
    'One-class SVM (OCSVM)': OCSVM(contamination=outliers_fraction),
    'Principal Component Analysis (PCA)': PCA(
        contamination=outliers_fraction, random_state=random_state),
    # 'Stochastic Outlier Selection (SOS)': SOS(
    #     contamination=outliers_fraction),
    # 'Locally Selective Combination (LSCP)': LSCP(
    #     detector_list, contamination=outliers_fraction,
    #     random_state=random_state),
    # 'Connectivity-Based Outlier Factor (COF)':
    #     COF(n_neighbors=35, contamination=outliers_fraction),
    # 'Subspace Outlier Detection (SOD)':
    #     SOD(contamination=outliers_fraction),
}


In [46]:
# Show all detectors
for i, clf in enumerate(classifiers.keys()):
    print('Model', i + 1, clf)

Model 1 Cluster-based Local Outlier Factor (CBLOF)
Model 2 Feature Bagging
Model 3 Histogram-base Outlier Detection (HBOS)
Model 4 Isolation Forest
Model 5 K Nearest Neighbors (KNN)
Model 6 Average KNN
Model 7 Local Outlier Factor (LOF)
Model 8 Minimum Covariance Determinant (MCD)
Model 9 One-class SVM (OCSVM)
Model 10 Principal Component Analysis (PCA)


In [47]:
# Fit the models with the generated data and
# compare model performances

# Fit the model
for i, (clf_name, clf) in enumerate(classifiers.items()):
    print()
    print(i + 1, 'fitting', clf_name)
    # fit the data and tag outliers
    clf.fit(x_train)

    # get the prediction labels and outlier scores of the training data
    y_train_pred = clf.labels_  # binary labels (0: inliers, 1: outliers)
    y_train_scores = clf.decision_scores_  # raw outlier scores

    # get the prediction on the test data
    y_test_pred = clf.predict(x_test)  # outlier labels (0 or 1)
    y_test_scores = clf.decision_function(x_test)  # outlier scores


    threshold = percentile(y_train_scores, 100 * outliers_fraction)

    n_errors = (y_train_pred != ground_truth).sum()

    # evaluate and print the results
    print("\nOn Training Data:")
    data.evaluate_print(clf_name, y_train, y_train_scores)
    print("\nOn Test Data:")
    data.evaluate_print(clf_name, y_test, y_test_scores)

    print("errors on training set = ", n_errors)

    print("threshold : ", threshold)


1 fitting Cluster-based Local Outlier Factor (CBLOF)

On Training Data:
Cluster-based Local Outlier Factor (CBLOF) ROC:0.3034, precision @ rank n:0.17

On Test Data:
Cluster-based Local Outlier Factor (CBLOF) ROC:0.3086, precision @ rank n:0.2014
errors on training set =  7915
threshold :  0.5208859281813201

2 fitting Feature Bagging

On Training Data:
Feature Bagging ROC:0.3935, precision @ rank n:0.2228

On Test Data:
Feature Bagging ROC:0.383, precision @ rank n:0.2491
errors on training set =  7455
threshold :  0.9986547832010001

3 fitting Histogram-base Outlier Detection (HBOS)

On Training Data:
Histogram-base Outlier Detection (HBOS) ROC:0.7524, precision @ rank n:0.0539

On Test Data:
Histogram-base Outlier Detection (HBOS) ROC:0.7672, precision @ rank n:0.0754
errors on training set =  8040
threshold :  32.381672173253875

4 fitting Isolation Forest

On Training Data:
Isolation Forest ROC:0.7786, precision @ rank n:0.1592

On Test Data:
Isolation Forest ROC:0.7887, precisio